In [1]:
using Qaintessent
using Qaintensor
using Qaintensor: optimize_contraction_order!, network_graph, line_graph, tree_decomposition
using BenchmarkTools

┌ Info: Precompiling Qaintensor [c26a0288-a22f-4b8a-8630-7ac49e34be6b]
└ @ Base loading.jl:1278


# Optimizing contraction ordering for expectation-value-like TensorNetworks

In [Markov & Shi, Simulating quantum computation by contracting tensor networks](https://arxiv.org/abs/quant-ph/0511069.) they study tensor networks that start with a product state and end with the projection onto another product state, as in the example below

```
1 □—————————————————□———————————□
                    |
2 □———————————□—————□—————□—————□
              |           |
3 □—————□———————————————————————□
        |     |           |
4 □—————□—————□———————————□—————□
```

These networks are the kind emerging from computing expectation values when starting with a product state. We see a great time saving also for MPS type circuits, **as long as no leg remains uncontracted** (for example, the output is projected into another MPS). The size of all legs of the original tensor should be roughly the same for getting good performance (no different orders of magnitude)

They prove that the contraction complexity for tensor networks with no open legs is <i>exponential in the treewidth of the underlying graph</i>, and give an algorithm for constructing an near-optimal contraction ordering given a near-optimal tree decomposition of the network graph. 

Cases when substantial improvement is achieved are:

* <i>Circuits of logarithmic depth</i>: If the depth of the circuit is logarithmic in the number of gates $T$, the complexity is polynomial in $T$
* <i>Circuits of local interacting qubits</i>: The complexity goes with $\exp(\sqrt{r}\sqrt{T})$, with $r$ the maximum distance between iteracting qubits and $T$ the total number of gates.

Finding the optimal tree decomposition of a graph is NP-complete, but some heuristics perform reasonably well in finding a near-optimal one. We have implemented the _minimum fill-in_ heuristic to provide the tree decomposition. For optimizing the contraction order just apply `optimize_contraction_order!` to a tensor network.

Below —after some setup code— you can find some benchmarking for typical circuits

In [2]:
# some setup
# copied from MPS branch
function ClosedMPS(T::AbstractVector{Tensor})
    l = length(T)
    @assert ndims(T[1]) == 2
    for i in 2:l-1
        @assert ndims(T[i]) == 3
    end
     @assert ndims(T[l]) == 2

    contractions = [Summation([1 => 2, 2 => 1]); [Summation([i => 3,i+1 => 1]) for i in 2:l-1]]
    openidx = reverse([1 => 1; [i => 2 for i in 2:l]])
    tn = TensorNetwork(T, contractions, openidx)
    return tn
end

Base.ndims(T::Tensor) = ndims(T.data)
Base.copy(net::TensorNetwork) = TensorNetwork(copy(net.tensors), copy(net.contractions), copy(net.openidx))

function shift_summation(S::Summation, step)
   return Summation([S.idx[i].first + step => S.idx[i].second for i in 1:2])
end

crand(dims...) = rand(ComplexF64, dims...)

# generate tensor network for benchmarking
""" Compute the expectation value of a random MPS when run through circuit `cgc`"""
function expectation_value(cgc::CircuitGateChain{N}; is_decompose = false) where N
    
    tensors = Tensor.([crand(2,2), [crand(2,2,2) for i in 2:N-1]..., crand(2,2)])
    T0 = ClosedMPS(tensors)
    
    T = copy(T0)
    tensor_circuit!(T, cgc, is_decompose = is_decompose)
    
    # measure
    T.contractions = [T.contractions; shift_summation.(T0.contractions, length(T.tensors))]
    for i in 1:N
        push!(T.tensors, T0.tensors[N+1-i])
        push!(T.contractions, Summation([T.openidx[end], (length(T.tensors) => T0.openidx[N+1-i].second)]))
        pop!(T.openidx)
    end
    T, cgc
end


# check that the new contraction order is valid (possible test)
N = 20
cgc = qft_circuit(N)
TN0, _ = expectation_value(cgc)
TN = copy(TN0)
optimize_contraction_order!(TN)

@assert TN.tensors == TN0.tensors
@assert TN.openidx == TN0.openidx
@assert Set(TN.contractions) == Set(TN0.contractions)

# Benchmarking the contraction order that `optimize_contraction_order!` produces

For each tensor network we have the following workflow

1. Benchmark the normal contraction
2. Optimize the contraction order and benchmark again the contraction
3. Benchmark the whole workflow (copy + optimize contraction order + contraction) 

#  QFT

In [3]:
N = 20

cgc = qft_circuit(N)
 
T0, _ = expectation_value(cgc);
contract(T0); # contract once to compile some of the functions 

In [4]:
T = copy(T0)
@benchmark contract($T)

BenchmarkTools.Trial: 
  memory estimate:  6.90 GiB
  allocs estimate:  89151
  --------------
  minimum time:     3.076 s (21.43% GC)
  median time:      3.119 s (21.08% GC)
  mean time:        3.119 s (21.08% GC)
  maximum time:     3.163 s (20.74% GC)
  --------------
  samples:          2
  evals/sample:     1

In [5]:
optimize_contraction_order!(T)
@benchmark contract($T)

BenchmarkTools.Trial: 
  memory estimate:  208.76 MiB
  allocs estimate:  36875
  --------------
  minimum time:     228.608 ms (7.19% GC)
  median time:      257.525 ms (12.67% GC)
  mean time:        282.472 ms (20.24% GC)
  maximum time:     624.159 ms (71.90% GC)
  --------------
  samples:          18
  evals/sample:     1

In [6]:
function copy_and_optimize(T0)
    T = copy(T0)
    optimize_contraction_order!(T)
    contract(T)
end

@benchmark copy_and_optimize($T0)

BenchmarkTools.Trial: 
  memory estimate:  511.85 MiB
  allocs estimate:  3279901
  --------------
  minimum time:     594.870 ms (12.52% GC)
  median time:      873.888 ms (37.49% GC)
  mean time:        848.234 ms (35.65% GC)
  maximum time:     990.146 ms (42.31% GC)
  --------------
  samples:          6
  evals/sample:     1

# Testing with `is_decompose = true`

In [7]:
N = 20

cgc = qft_circuit(N)
 
T0, _ = expectation_value(cgc, is_decompose = true);

In [8]:
T = copy(T0)
@benchmark contract($T)

BenchmarkTools.Trial: 
  memory estimate:  31.64 GiB
  allocs estimate:  167578
  --------------
  minimum time:     26.174 s (18.71% GC)
  median time:      26.174 s (18.71% GC)
  mean time:        26.174 s (18.71% GC)
  maximum time:     26.174 s (18.71% GC)
  --------------
  samples:          1
  evals/sample:     1

In [9]:
optimize_contraction_order!(T)
@benchmark contract($T)

BenchmarkTools.Trial: 
  memory estimate:  142.09 MiB
  allocs estimate:  61949
  --------------
  minimum time:     150.826 ms (6.72% GC)
  median time:      217.194 ms (12.04% GC)
  mean time:        221.207 ms (9.87% GC)
  maximum time:     329.340 ms (8.00% GC)
  --------------
  samples:          23
  evals/sample:     1

In [10]:
@benchmark copy_and_optimize($T0)

BenchmarkTools.Trial: 
  memory estimate:  688.75 MiB
  allocs estimate:  6343072
  --------------
  minimum time:     767.063 ms (15.29% GC)
  median time:      851.170 ms (14.97% GC)
  mean time:        846.029 ms (15.11% GC)
  maximum time:     907.540 ms (13.91% GC)
  --------------
  samples:          6
  evals/sample:     1

# Testing adder without decompose

In [11]:
N = 7 # total qubits = 22
cgc = vbe_adder_circuit(N)
T0, _ = expectation_value(cgc);

In [12]:
T = copy(T0)
@benchmark contract($T)

BenchmarkTools.Trial: 
  memory estimate:  6.25 GiB
  allocs estimate:  30905
  --------------
  minimum time:     5.277 s (28.22% GC)
  median time:      5.277 s (28.22% GC)
  mean time:        5.277 s (28.22% GC)
  maximum time:     5.277 s (28.22% GC)
  --------------
  samples:          1
  evals/sample:     1

In [13]:
optimize_contraction_order!(T)
@benchmark contract($T)

BenchmarkTools.Trial: 
  memory estimate:  567.12 MiB
  allocs estimate:  14568
  --------------
  minimum time:     776.808 ms (7.45% GC)
  median time:      834.194 ms (6.95% GC)
  mean time:        900.369 ms (15.59% GC)
  maximum time:     1.151 s (34.67% GC)
  --------------
  samples:          6
  evals/sample:     1

In [14]:
@benchmark copy_and_optimize($T0)

BenchmarkTools.Trial: 
  memory estimate:  621.31 MiB
  allocs estimate:  581792
  --------------
  minimum time:     814.420 ms (7.45% GC)
  median time:      905.871 ms (8.26% GC)
  mean time:        951.305 ms (14.63% GC)
  maximum time:     1.210 s (19.17% GC)
  --------------
  samples:          6
  evals/sample:     1

# With decompose

In [15]:
N = 7 # total qubits = 22
cgc = vbe_adder_circuit(N)
T0, _ = expectation_value(cgc, is_decompose = true);

In [16]:
T = copy(T0)
@benchmark contract($T)

BenchmarkTools.Trial: 
  memory estimate:  46.82 GiB
  allocs estimate:  63580
  --------------
  minimum time:     28.811 s (22.24% GC)
  median time:      28.811 s (22.24% GC)
  mean time:        28.811 s (22.24% GC)
  maximum time:     28.811 s (22.24% GC)
  --------------
  samples:          1
  evals/sample:     1

In [17]:
optimize_contraction_order!(T)
@benchmark contract($T)

BenchmarkTools.Trial: 
  memory estimate:  858.94 MiB
  allocs estimate:  21292
  --------------
  minimum time:     1.570 s (10.84% GC)
  median time:      1.599 s (10.65% GC)
  mean time:        1.740 s (17.23% GC)
  maximum time:     2.052 s (29.72% GC)
  --------------
  samples:          3
  evals/sample:     1

In [18]:
@benchmark copy_and_optimize($T0)

BenchmarkTools.Trial: 
  memory estimate:  955.28 MiB
  allocs estimate:  1107967
  --------------
  minimum time:     1.754 s (12.85% GC)
  median time:      1.904 s (19.18% GC)
  mean time:        1.864 s (19.19% GC)
  maximum time:     1.935 s (24.94% GC)
  --------------
  samples:          3
  evals/sample:     1

# Testing another adder

In [19]:
N = 7 # total qubits = 22
cgc = qcla_inplace_adder_circuit(N)
T0, _ = expectation_value(cgc);

In [20]:
T = copy(T0)
@benchmark contract($T)

BenchmarkTools.Trial: 
  memory estimate:  17.50 GiB
  allocs estimate:  40758
  --------------
  minimum time:     12.767 s (24.45% GC)
  median time:      12.767 s (24.45% GC)
  mean time:        12.767 s (24.45% GC)
  maximum time:     12.767 s (24.45% GC)
  --------------
  samples:          1
  evals/sample:     1

In [21]:
optimize_contraction_order!(T)
@benchmark contract($T)

BenchmarkTools.Trial: 
  memory estimate:  552.77 MiB
  allocs estimate:  19702
  --------------
  minimum time:     1.029 s (3.44% GC)
  median time:      1.089 s (6.81% GC)
  mean time:        1.162 s (15.63% GC)
  maximum time:     1.387 s (32.07% GC)
  --------------
  samples:          5
  evals/sample:     1

In [22]:
@benchmark copy_and_optimize($T0)

BenchmarkTools.Trial: 
  memory estimate:  638.12 MiB
  allocs estimate:  838869
  --------------
  minimum time:     1.146 s (5.22% GC)
  median time:      1.322 s (17.06% GC)
  mean time:        1.332 s (18.08% GC)
  maximum time:     1.538 s (29.41% GC)
  --------------
  samples:          4
  evals/sample:     1